In [ ]:
#Notebook description / Structure

#this notebook is intended to analyze the broader economy as a whole and give a sense of the macroeconomic 
#headwinds that will effect the asset and labor markets. 

# The first section will be a brief overview of GDP which is the broadest measure of economic activity. and a deep analysis of gdp including
# the components of 
#   * GDP = 
#   * Consumption+
#   * Investment+
#   * Government Spending+
#   * Net Exports (Exports - Imports) 

In [ ]:
#TODOS 
# plot the components of GDP which are
# 1. Consumption
# 2. Investment
# 3. Government Spending (current consumption and investment in capital goods)
# 4. Net Exports (Exports - Imports)

# get real gdp growht rate and nominal gdp growth rate
# get the gdp deflator

# work on visualzing and analyzing the
# the circular flow of income and expenditure 
# three principal markets
# 1. goods and services market
# 2. labor market
# 3. (capital) / financial markets


In [ ]:
#LOAD: libararies, parameters, functions

from cmath import nan
from datetime import date
#import nasdaqdatalink
#nasdaqdatalink.ApiConfig.api_key = "4vgocb_MAow2x5-Xm5dv"
import copy


import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
from   plotly.subplots import make_subplots
import nasdaqdatalink
from sklearn.preprocessing import MinMaxScaler
from Quantapp.EconomicData import EconomicData


plt.rcParams["figure.figsize"] = (10, 2)

ed = EconomicData()


In [ ]:

fred_key = 'c2cfb2de66da3f050ea4e32d3f5ffcf8'

gdp_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=GDP&api_key={fred_key}&file_type=json'
real_gdp_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=GDPC1&api_key={fred_key}&file_type=json'
per_capita_real_gdp_query =  f'https://api.stlouisfed.org/fred/series/observations?series_id=A939RX0Q048SBEA&api_key={fred_key}&file_type=json'
ppi_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=PPIACO&api_key={fred_key}&file_type=json'
cpi_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=CPIAUCNS&api_key={fred_key}&file_type=json'
interest_rate_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key={fred_key}&file_type=json'
unemployment_rate_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=UNRATE&api_key={fred_key}&file_type=json'

nber_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=USREC&api_key={fred_key}&file_type=json'
oecd_query = f'https://api.stlouisfed.org/fred/series/observations?series_id=USARECDM&api_key={fred_key}&file_type=json'
real_time_sahm_rule = f'https://api.stlouisfed.org/fred/series/observations?series_id=SAHMREALTIME&api_key={fred_key}&file_type=json'
markov_switching_smoothed = f'https://api.stlouisfed.org/fred/series/observations?series_id=RECPROUSM156N&api_key={fred_key}&file_type=json'

def fetch_fred_json(query):
    """
    Fetches JSON data from the provided FRED API query URL.
    
    Args:
        query (str): The FRED API query URL.
    
    Returns:
        dict: Parsed JSON response from the API.
        
    Raises:
        requests.exceptions.HTTPError: If the HTTP request returned an unsuccessful status code.
        requests.exceptions.RequestException: For other request-related errors.
    """
    import requests
    
    try:
        response = requests.get(query)
        response.raise_for_status()  # Raise an error for bad status codes
        df = pd.DataFrame(response.json()['observations']).drop(columns=['realtime_start', 'realtime_end'])
        #create a new dataframe where index is a datetime object and the value is the observation value, drop the
        value = pd.to_numeric(df['value'], errors='coerce')
        index = pd.to_datetime(df['date'])
        df    = pd.DataFrame(value.values, index=index, columns=['value'])        
        return df
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Python 3.6+
        raise
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
        raise

def plot_cpi_ppi_on_same_graph(cpi_df, ppi_df, nber_df, title="CPI vs PPI Comparison with NBER Recessions", x_label="Date", y_label="Index Value"):
    """
    Plots CPI and PPI on the same graph using Plotly for comparison and highlights NBER recession periods.
    
    Args:
        cpi_df (pd.DataFrame): DataFrame containing CPI data with datetime index and 'value' column.
        ppi_df (pd.DataFrame): DataFrame containing PPI data with datetime index and 'value' column.
        nber_df (pd.DataFrame): DataFrame containing NBER recession data with datetime index and 'value' column.
        title (str): Title of the plot.
        x_label (str): Label for the x-axis.
        y_label (str): Label for the y-axis.
    
    Returns:
        None: Displays the interactive Plotly plot.
    """
    import plotly.graph_objects as go
    import pandas as pd
    from itertools import groupby
    from operator import itemgetter

    # Rename 'value' columns for clarity
    cpi_df_renamed = cpi_df.rename(columns={'value': 'CPI'})
    ppi_df_renamed = ppi_df.rename(columns={'value': 'PPI'})
    nber_df_renamed = nber_df.rename(columns={'value': 'Recession'})

    # Merge CPI and PPI DataFrames on the Date index
    merged_df = pd.merge(cpi_df_renamed, ppi_df_renamed, left_index=True, right_index=True, how='inner')

    # Merge with NBER DataFrame
    merged_df = pd.merge(merged_df, nber_df_renamed, left_index=True, right_index=True, how='left').fillna(0)

    # Identify recession periods
    recession_periods = []
    in_recession = False
    start_date = None

    for date, group in merged_df['Recession'].groupby(merged_df.index):
        if group.iloc[0] == 1 and not in_recession:
            in_recession = True
            start_date = date
        elif group.iloc[0] == 0 and in_recession:
            in_recession = False
            end_date = date
            recession_periods.append((start_date, end_date))

    # Handle if the last period is still in recession
    if in_recession:
        recession_periods.append((start_date, merged_df.index[-1]))

    # Initialize Plotly figure
    fig = go.Figure()

    # Add CPI trace
    fig.add_trace(go.Scatter(
        x=merged_df.index,
        y=merged_df['CPI'],
        mode='lines',
        name='CPI',
        line=dict(color='blue')
    ))

    # Add PPI trace
    fig.add_trace(go.Scatter(
        x=merged_df.index,
        y=merged_df['PPI'],
        mode='lines',
        name='PPI',
        line=dict(color='red', dash='dash')
    ))

    # Add recession shaded regions
    for period in recession_periods:
        fig.add_vrect(
            x0=period[0],
            x1=period[1],
            fillcolor="grey",
            opacity=0.3,
            layer="below",
            line_width=0
        )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        template='plotly_dark',
        hovermode='x unified',
        legend=dict(x=0, y=1),
        height=600
    )

    # Show figure
    fig.show()

def plot_dataframe_with_recession_shading(
    data_df,
    nber_df,
    title="Data with NBER Recessions",
    x_label="Date",
    y_label="Value"
):
    """
    Plots each column in 'data_df' as a separate time-series line and highlights NBER recession periods.

    Args:
        data_df (pd.DataFrame): A DataFrame with one or more columns representing time-series data.
                                Must have a DateTimeIndex.
        nber_df (pd.DataFrame): A DataFrame with a 'value' column indicating recessions (1) or no recessions (0),
                                also indexed by DateTime.
        title (str): Title of the plot.
        x_label (str): Label for the x-axis.
        y_label (str): Label for the y-axis.

    Returns:
        None: Displays the interactive Plotly plot.
    """
    import plotly.graph_objects as go
    import pandas as pd

    # Ensure columns are labels for clarity in the legend
    # Copy the data so we don't overwrite anything
    df_copy = data_df.copy()

    # Rename NBER's 'value' column to 'Recession'
    nber_df_renamed = nber_df.rename(columns={"value": "Recession"})
    
    # Merge the main data with the NBER data for recession info
    merged_df = pd.merge(
        df_copy,
        nber_df_renamed,
        left_index=True,
        right_index=True,
        how="left"
    ).fillna({"Recession": 0})

    # Identify recession periods
    recession_periods = []
    in_recession = False
    start_date = None

    for date, group in merged_df["Recession"].groupby(merged_df.index):
        if group.iloc[0] == 1 and not in_recession:
            in_recession = True
            start_date = date
        elif group.iloc[0] == 0 and in_recession:
            in_recession = False
            end_date = date
            recession_periods.append((start_date, end_date))

    # If the last period is still in recession, close it at the final date
    if in_recession:
        recession_periods.append((start_date, merged_df.index[-1]))

    # Initialize Plotly figure
    fig = go.Figure()

    # Plot each column (excluding 'Recession')
    # Assign each column a different color or line style as needed
    colors = [
        "blue", "red", "green", "orange", "purple", "brown",
        "pink", "olive", "cyan", "gray"
    ]
    line_styles = ["solid", "dash", "dot", "dashdot"]

    col_list = [col for col in merged_df.columns if col != "Recession"]
    for idx, col in enumerate(col_list):
        fig.add_trace(
            go.Scatter(
                x=merged_df.index,
                y=merged_df[col],
                mode="lines",
                name=col,
                line=dict(
                    color=colors[idx % len(colors)],
                    dash=line_styles[idx % len(line_styles)]
                ),
            )
        )

    # Add recession shaded regions
    for period in recession_periods:
        fig.add_vrect(
            x0=period[0],
            x1=period[1],
            fillcolor="grey",
            opacity=0.3,
            layer="below",
            line_width=0
        )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        template="plotly_dark",
        hovermode="x unified",
        legend=dict(x=0, y=1),
        height=600
    )

    # Show figure
    fig.show()

gdp_df = fetch_fred_json(gdp_query)
real_gdp_df = fetch_fred_json(real_gdp_query)
per_capita_real_gdp_query_df = fetch_fred_json(per_capita_real_gdp_query)

unemployment_rate_df = fetch_fred_json(unemployment_rate_query)
ppi_df = fetch_fred_json(ppi_query).rename(columns={'value': 'PPI'})
cpi_df = fetch_fred_json(cpi_query).rename(columns={'value': 'CPI'})
cpi_ppi_df = pd.concat([cpi_df, ppi_df], axis=1)
cpi_ppi_df_3month_percent_change = cpi_ppi_df.pct_change(3).dropna().rename(columns={'CPI': 'CPI 3-Month % Change', 'PPI': 'PPI 3-Month % Change'})
cpi_ppi_df_6month_percent_change = cpi_ppi_df.pct_change(6).dropna().rename(columns={'CPI': 'CPI 6-Month % Change', 'PPI': 'PPI 6-Month % Change'})
cpi_ppi_df_12month_percent_change = cpi_ppi_df.pct_change(12).dropna().rename(columns={'CPI': 'CPI 12-Month % Change', 'PPI': 'PPI 12-Month % Change'})
cpi_ppi_df_24month_percent_change = cpi_ppi_df.pct_change(24).dropna().rename(columns={'CPI': 'CPI 24-Month % Change', 'PPI': 'PPI 24-Month % Change'})
cpi_ppi_df_36month_percent_change = cpi_ppi_df.pct_change(36).dropna().rename(columns={'CPI': 'CPI 36-Month % Change', 'PPI': 'PPI 36-Month % Change'})
combined_cpi_ppi_df = pd.concat([cpi_ppi_df_3month_percent_change, cpi_ppi_df_6month_percent_change, cpi_ppi_df_12month_percent_change, cpi_ppi_df_24month_percent_change, cpi_ppi_df_36month_percent_change], axis=1)

cpi_minus_ppi = pd.DataFrame(
    (cpi_df['CPI'] - ppi_df['PPI']), 
    columns=['CPI - PPI']
)
#calculate the difference between cpi and ppi



#combine ppi and cpi dataframes, keep the name of the column as the name of the dataframe



interest_rate_df = fetch_fred_json(interest_rate_query)

nber_df = fetch_fred_json(nber_query)
oecd_df = fetch_fred_json(oecd_query)
real_time_sahm_rule_df = fetch_fred_json(real_time_sahm_rule)
markov_switching_smoothed_df = fetch_fred_json(markov_switching_smoothed)

#plot the data
import matplotlib.pyplot as plt
import pandas as pd


#create a plotly function that will plot the data
def plotly_line(df, title, x_label, y_label):
    import plotly.express as px
    fig = px.line(df, title=title, labels={df.index.name: x_label, 'value': y_label})
    return fig

print(unemployment_rate_df)

In [ ]:
plot_dataframe_with_recession_shading(gdp_df, nber_df, title="GDP with NBER Recessions", y_label="GDP")
plot_dataframe_with_recession_shading(real_gdp_df, nber_df, title="Real GDP with NBER Recessions", y_label="Real GDP")
plot_dataframe_with_recession_shading(per_capita_real_gdp_query_df, nber_df, title="Per Capita Real GDP with NBER Recessions", y_label="Per Capita Real GDP")


In [ ]:
plot_cpi_ppi_on_same_graph(cpi_df, ppi_df, nber_df)
plot_dataframe_with_recession_shading(cpi_minus_ppi, nber_df, title="CPI - PPI with NBER Recessions", x_label="Date", y_label="Index Value")

In [ ]:
plot_dataframe_with_recession_shading(interest_rate_df, nber_df, title="Interest Rate with NBER Recessions", y_label="Interest Rate")

In [ ]:
plot_dataframe_with_recession_shading(unemployment_rate_df, nber_df, title="Unemployment Rate with NBER Recessions", y_label="Unemployment Rate")
